# Parallel Work: Threads and Processes

When to use threads vs processes, how to handle shared state, and practical examples using `concurrent.futures`.

In [17]:
import os
import time
import math
import threading
from queue import Queue
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor

## Choosing Threads vs Processes
- Python threads share memory and are great for I/O-bound work (waiting on disk/network). The GIL prevents multiple threads from running Python bytecode simultaneously.
- Processes sidestep the GIL for CPU-bound work; they do not share memory, so use picklable arguments/results.
- Rule of thumb: I/O → threads; CPU → processes.

In [18]:
print(f"Logical CPUs: {os.cpu_count()}")

Logical CPUs: 24


## I/O-Bound Work with Threads
Use `ThreadPoolExecutor` when tasks mostly wait on I/O (network, disk, sleep).

In [19]:
def fake_download(idx: int) -> str:
    time.sleep(0.4)  # simulate waiting on network
    return f"item-{idx}"

start = time.perf_counter()
with ThreadPoolExecutor(max_workers=5) as pool:
    for result in pool.map(fake_download, range(10)):
        print("received", result)
elapsed = time.perf_counter() - start
print(f"Threaded I/O finished in {elapsed:.2f}s")

received item-0
received item-1
received item-2
received item-3
received item-4
received item-5
received item-6
received item-7
received item-8
received item-9
Threaded I/O finished in 0.80s
received item-5
received item-6
received item-7
received item-8
received item-9
Threaded I/O finished in 0.80s


## CPU-Bound Work with Processes
`ProcessPoolExecutor` bypasses the GIL. Use picklable callables/arguments. Guard with `if __name__ == "__main__"` on Windows.

In [24]:
import multiprocessing


def is_prime(n: int) -> bool:
    if n < 2:
        return False
    if n % 2 == 0:
        return n == 2
    limit = int(math.sqrt(n)) + 1
    for i in range(3, limit, 2):
        if n % i == 0:
            return False
    return True

def count_primes_up_to(n: int) -> int:
    # add a tiny delay to keep processes busy for a notable amount of time
    time.sleep(0.05)
    return sum(1 for x in range(n) if is_prime(x))


def run_prime_pool(workers: int | None = None):
    workers = workers or os.cpu_count()
    inputs = [150_000 + i * 5_000 for i in range(workers)]
    start = time.perf_counter()
    ctx = multiprocessing.get_context("spawn")
    with ProcessPoolExecutor(max_workers=workers, mp_context=ctx) as pool:
        counts = list(pool.map(count_primes_up_to, inputs))
    elapsed = time.perf_counter() - start
    for n, count in zip(inputs, counts):
        print(f"Primes below {n:,}: {count}")
    print(f"Process pool completed in {elapsed:.2f}s using {workers} workers")


if __name__ == "__main__":
    if "get_ipython" in globals():
        print("Process pool demo is meant to run from a fresh kernel or CLI; restart kernel then call run_prime_pool(), or run `python 04.parallel.py`.")
    else:
        multiprocessing.freeze_support()
        multiprocessing.set_start_method("spawn", force=True)
        run_prime_pool()

Process pool demo is meant to run from a fresh kernel or CLI; restart kernel then call run_prime_pool(), or run `python 04.parallel.py`.


## Shared State and Locks (Threads)
Threads share memory. Use locks to protect shared state; prefer message passing (queues) when possible.

In [21]:
counter = 0
lock = threading.Lock()

def add_many(n: int):
    global counter
    for _ in range(n):
        with lock:
            counter += 1

threads = [threading.Thread(target=add_many, args=(100_000,)) for _ in range(5)]
start = time.perf_counter()
for t in threads:
    t.start()
for t in threads:
    t.join()
elapsed = time.perf_counter() - start
print(f"Counter = {counter} (expected 500000)")
print(f"Locked increments completed in {elapsed:.2f}s")

Counter = 500000 (expected 500000)
Locked increments completed in 0.07s


## Producer/Consumer with Queue
Use a `Queue` to safely hand off work between threads without manual locks.

In [22]:
q: Queue[str] = Queue()
results = []

# Producer
for i in range(5):
    q.put(f"task-{i}")

# Consumers

def worker(name: str):
    while True:
        try:
            item = q.get_nowait()
        except Exception:
            break
        time.sleep(0.1)
        results.append((name, item))
        q.task_done()

threads = [threading.Thread(target=worker, args=(f"worker-{i}",)) for i in range(3)]
for t in threads:
    t.start()
for t in threads:
    t.join()

print("Results (unordered):")
for entry in results:
    print(entry)

Results (unordered):
('worker-2', 'task-2')
('worker-1', 'task-1')
('worker-0', 'task-0')
('worker-2', 'task-3')
('worker-1', 'task-4')


## Summary
- Threads excel at I/O-bound tasks; processes excel at CPU-bound tasks.
- Protect shared state with locks or, better, avoid sharing via queues.
- For CPU pools on Windows, keep pool creation under `if __name__ == "__main__"`.
- Keep functions small and picklable for process pools.